In [244]:
import pandas as pd
import numpy as np
import os
import Dijkstra_v2 as dijk   # 다익스트라 모듈 (지난 과제 활용)

In [253]:
def make_dataframe_dijk(graph_file):
    dist_df = pd.DataFrame()
    graph,node_number = dijk.read_graph(graph_file)   # 그래프 읽어오기//텍스트 형식의 인접 리스트 넣어주면 그래프와 노드 수 가져옴      
    for start in range(node_number):                          # 최단거리 nxn matrix 가져오기.
        dist_df[start] = dijk.find_shortest_path(graph,start)      # 다익스트라 코드 재활용하여 반복함.
    dist_df.columns = [chr(i+65) for i in np.array(dist_df.columns)]  # 노드명 숫자로 되어있는데 알파벳으로 변환
    dist_df.index = dist_df.columns
    return dist_df, node_number

graph_file = './graph_figure6-11.txt'
node_df = pd.read_excel('./UFCLP_nodeInfoTable.xlsx', index_col = 0)

dist_df, node_number = make_dataframe_dijk(graph_file)  ## shortest table 12x12 , node number 12
demand_df = node_df.iloc[:,0]  ##demand, 12x1
fixed_cost_df = node_df.iloc[:,1]  ##fixed cost, 12x1

# 초기 lambda 값 정의
lambda_1st = 10*(demand_df.mean()+fixed_cost_df.mean())
lambda_df = np.array([lambda_1st for _ in range(node_number)])
print("1st lambda_i:",lambda_df)

#alpha = distance to cost : 0.35
alpha = 0.35

#lagrangian alpha value
lag_alpha = 2 


def find_V_df():
    V_df = pd.DataFrame(index=dist_df.index, columns = dist_df.columns)  #빈 데이터프레임 생성.
    # V Dataframe
    for j in range(node_number):
        d_j = np.array(dist_df.iloc[:,j]).ravel()
        dem_dist = d_j*alpha*demand_df-lambda_df
        # print(array)
        # np.where(array<0,array,0).sum()
        V_df.iloc[:,j] = np.where(dem_dist<0,dem_dist,0)

    return V_df

def find_V_j():
    # V_j
    V_j = V_df.sum() + np.array(fixed_cost_df).ravel()
    # print("V_j", V_j, "type of V_j is :",type(V_j))
    return V_j

def find_X_j():
    # X_j
    X_j = np.where(V_j<0,1,0)
    return X_j

def find_Y_ij():
    # Y_ij
    Y_ij = np.where((V_df<0)&(X_j==1),1,0)
    return Y_ij

def find_LB():
#LB (relaxed solution)
    sigma_fX = np.dot(np.array(fixed_cost_df).ravel(),X_j)
    double_sigma_ahd = 0 ## 초기화 필요
    for i in range(node_number):
        d_j = np.array(dist_df.iloc[i,:]).ravel()
        Y_j = Y_ij[i,:]
        demand_dist = (alpha*demand_df[i]*d_j-lambda_df[i])*Y_j
        double_sigma_ahd += demand_dist.sum()

    LB = sigma_fX + double_sigma_ahd + lambda_df.sum()  #sigma(fX)
    # print("LB:",LB)
    return LB
#-----------------



def find_UB():
#UB (feasible solution)
    sigma_fX = np.dot(np.array(fixed_cost_df).ravel(),X_j)
    double_sigma_ahd = 0 ## 초기화 필요
    for i in range(node_number):
        d_j = np.array(dist_df.iloc[i,:]).ravel()
        Y_j = Y_ij[i,:]
        demand_dist = alpha*demand_df[i]*d_j*Y_j
        double_sigma_ahd += demand_dist.sum()

    UB = sigma_fX + double_sigma_ahd + lambda_df.sum()
    # print("UB:", UB)
    return UB

def find_stepsize():
#update stepsize
    divider = 0
    for i in range(node_number):
        sigma_Y_j = (Y_ij[i,:].sum() - 1)**2
        divider += sigma_Y_j

    t_n = lag_alpha*(UB-LB)/divider
    # print("t_n:",t_n)
    
    return t_n #scalar

#update lambda_
def update_lambda(lambda_df):
    violation = Y_ij.sum(axis=1) - 1
    # print(lambda_df-t_n*violation)
    lambda_df = np.where(lambda_df-t_n*violation > 0, lambda_df-t_n*violation,0) 
    # print("updated lambda_df:",lambda_df)
    return lambda_df

def check_bound_better():
    temp_t_n = find_stepsize()
    temp_lambda_df = update_lambda(lambda_df)
    temp_V_df = find_V_df()
    temp_V_j = find_V_j()
    temp_X_j = find_X_j()
    temp_Y_ij = find_Y_ij()
    temp_UB = find_UB()
    temp_LB = find_LB()
    return temp_t_n, temp_lambda_df, temp_V_df, temp_V_j, temp_X_j, temp_Y_ij, temp_UB, temp_LB

#1st cycle
#------
# t_n = find_stepsize()
# lambda_df = update_lambda(lambda_df)
#-------

max_iteration = 4
iteration = 0
alpha_iteration = 0
  
#초기 시행
V_df = find_V_df()
V_j = find_V_j()
X_j = find_X_j()
Y_ij = find_Y_ij()
LB = find_LB()
UB = find_UB()

while abs(LB-UB) > 100 and lag_alpha > 0.1 and alpha_iteration<max_iteration:
  
    iteration += 1
    alpha_iteration = 0

    while True:     # alpha 값을 유지한 채 iteration을 계속 진행할지, 아니면 alpha를 바꿀지를 결정하는 단계
                                                
        alpha_iteration+=1
        temp_t_n, temp_lambda_df, temp_V_df, temp_V_j, temp_X_j, temp_Y_ij, temp_UB, temp_LB = check_bound_better()  # temp 값들로 bound 갱신
        print("{0}-{1}. LB가 좋아지지 않아 alpha를 갱신합니다.".format(iteration,alpha_iteration))
        print("현재 alpha 값은 {0} 입니다".format(lag_alpha))

        if (temp_LB > LB)|(lag_alpha<=0.1)|(alpha_iteration == max_iteration):
            break
        else:
            lag_alpha *=0.5      # LB값은 점점 커져야 좋아진다. 근데 안좋아지면, 알파값을 계속 절반으로 만드는 단계이다.

    # UB 좋아져서 while문 탈출한거니까 이때부턴 forward 진행
    t_n, lambda_df, V_df, V_j, X_j, Y_ij, UB,LB = temp_t_n, temp_lambda_df, temp_V_df, temp_V_j, temp_X_j, temp_Y_ij, temp_UB, temp_LB



    print("LB:{0},UB:{1:.3f},alpha:{2:.3f},t_n:{3:.3f}".format(LB,UB,lag_alpha,t_n))
    print("---------------------------------------------------")

1st lambda_i: [1916.66666667 1916.66666667 1916.66666667 1916.66666667 1916.66666667
 1916.66666667 1916.66666667 1916.66666667 1916.66666667 1916.66666667
 1916.66666667 1916.66666667]
LB: -224537.69999999998
UB: 51462.3
t_n: 380.1652892561983
UB: 51462.3
LB: -224537.69999999998
1-1. LB가 좋아지지 않아 alpha를 갱신합니다.
현재 alpha 값은 2 입니다
t_n: 190.08264462809916
UB: 51462.3
LB: -224537.69999999998
1-2. LB가 좋아지지 않아 alpha를 갱신합니다.
현재 alpha 값은 1.0 입니다
t_n: 95.04132231404958
UB: 51462.3
LB: -224537.69999999998
1-3. LB가 좋아지지 않아 alpha를 갱신합니다.
현재 alpha 값은 0.5 입니다
t_n: 47.52066115702479
UB: 51462.3
LB: -224537.69999999998
1-4. LB가 좋아지지 않아 alpha를 갱신합니다.
현재 alpha 값은 0.25 입니다
LB:-224537.69999999998,UB:51462.300,alpha:0.250,t_n:47.521
---------------------------------------------------
